# Generate representational similarity matrix

In [1]:
%matplotlib inline

# Handling files
import os
from os.path import join as opj
from glob import glob
import scipy.io as sio

# Analysis
import pandas as pd
import numpy as np

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

Helper function: Cosine distance

In [3]:
def cosine_distance(v1, v2):
    dotprod = np.dot(v1, v2)
    mag_v1 = np.sum(np.sqrt(np.power(v1, 2)))
    mag_v2 = np.sum(np.sqrt(np.power(v2, 2)))
    dist = dotprod/(mag_v1*mag_v2)
    
    return dist

Experiment directories:

In [31]:
scratch_dir = os.environ['PI_SCRATCH']
project_dir = opj(scratch_dir, 'OPUS')

# Model output: Self and other weights
sim_dir = opj(project_dir, 'sim_data')
sim_files = glob(opj(sim_dir, '*.mat'))
sim_files.sort()

# Test phase behavior
behav_dir = opj(project_dir, 'session_data')
test_files = glob(opj(behav_dir, '*test*'))
test_files.sort()

# Probe task behavior
probe_files = glob(opj(behav_dir, '*probe*'))
probe_files.sort()
test_files

['/scratch/groups/hyo/OPUS/session_data/sll_opusfmri_03.test.1.180816_1905.mat',
 '/scratch/groups/hyo/OPUS/session_data/sll_opusfmri_03.test.2.180816_1915.mat',
 '/scratch/groups/hyo/OPUS/session_data/sll_opusfmri_04.test.1.180822_1857.mat',
 '/scratch/groups/hyo/OPUS/session_data/sll_opusfmri_04.test.2.180822_1907.mat',
 '/scratch/groups/hyo/OPUS/session_data/sll_opusfmri_05.test.1.180823_1859.mat',
 '/scratch/groups/hyo/OPUS/session_data/sll_opusfmri_05.test.2.180823_1908.mat',
 '/scratch/groups/hyo/OPUS/session_data/sll_opusfmri_06.test.1.180825_1725.mat',
 '/scratch/groups/hyo/OPUS/session_data/sll_opusfmri_06.test.2.180825_1734.mat',
 '/scratch/groups/hyo/OPUS/session_data/sll_opusfmri_07.test.1.180827_1856.mat',
 '/scratch/groups/hyo/OPUS/session_data/sll_opusfmri_07.test.2.180827_1905.mat',
 '/scratch/groups/hyo/OPUS/session_data/sll_opusfmri_08.test.1.180829_1852.mat',
 '/scratch/groups/hyo/OPUS/session_data/sll_opusfmri_08.test.2.180829_1902.mat']

## Representational similarity matrices

### Read model-fitting data

In [29]:
f = sim_files[0]

# Load ground-truth weights from sim_data
tmp_data = sio.loadmat(f)
tmp_self = tmp_data['sim_agent']['orig_weights'][0][0]
tmp_self = tmp_self.flatten()
tmp_other = tmp_data['sim_agent']['sim_weights'][0][0]
tmp_other = tmp_other.flatten()



[ 0.31663004 -0.57546922  0.13034145]
[1.  0.5 0. ]


### RSM

## Compare to probe task behavior

## Scratch

In [ ]:
movies = {'[P, H, R]': [1, 1, 1],
         '[P, H, A]': [1, 1, -1],
         '[P, F, R]': [1, -1, 1],
         '[P, F, A]':[1, -1, -1],
         '[N, H, R]': [-1, 1, 1],
         '[N, H, A]': [-1, 1, -1],
         '[N, F, R]': [-1, -1, 1],
         '[N, F, A]': [-1, -1, -1]}

movie_df = pd.DataFrame([(m1, v1, m2, v2, cosine_distance(v1, v2))
                          for m1, v1 in movies.items() for m2, v2 in movies.items()],
                          columns = ['movie1', 'feat1', 'movie2', 'feat2', 'Features'])

In [ ]:
def draw_heatmap(*args, **kwargs):
    data = kwargs.pop('data')
    d = data.pivot(index=args[1], columns=args[0], values=args[2])
    ax = sns.heatmap(d, **kwargs)
    return ax

In [ ]:
self_preferences = [1, 0, 0]
other_preferences = [0, 1, 0]

In [ ]:
movie_df['self1'] = [np.multiply(f, self_preferences) for f in movie_df['feat1']]
movie_df['self2'] = [np.multiply(f, self_preferences) for f in movie_df['feat2']]
movie_df['Self'] = [cosine_distance(s1, s2) for s1, s2 in zip(movie_df['self1'], movie_df['self2'])]

In [ ]:
movie_df['other1'] = [np.multiply(f, other_preferences) for f in movie_df['feat1']]
movie_df['other2'] = [np.multiply(f, other_preferences) for f in movie_df['feat2']]
movie_df['Other'] = [cosine_distance(s1, s2) for s1, s2 in zip(movie_df['other1'], movie_df['other2'])]

In [ ]:
distance_df = movie_df.copy()
distance_df = distance_df[['movie1', 'movie2', 'Features', 'Self', 'Other']]
distance_df = pd.melt(distance_df, id_vars = ['movie1', 'movie2'],
                      value_vars=['Features', 'Self', 'Other'],
                      var_name='Reference frame',
                      value_name='Distance')
distance_df[:10]

In [ ]:
fg = sns.FacetGrid(distance_df, col='Reference frame', gridspec_kws={"wspace":0})
fg.map_dataframe(draw_heatmap, 'movie1', 'movie2', 'Distance', cmap='viridis', square = True)
for ax in fg.axes.flat:
    # set aspect of all axis
    ax.set(xlabel='', ylabel='')